In [ ]:
# for mooring, can we just interpolate the gridded fields directly rather than interpolating to points and then re-gridding?
# YES but it's slower (!?!)
X = ds.XC.isel(time=0) 
Y = ds.YC.isel(time=0)
i = ds.i
j = ds.j
z = ds.Z.isel(time=0)
k = ds.k
f_x = interpolate.interp1d(X[0,:].values, i)
f_y = interpolate.interp1d(Y[:,0].values, j)
f_z = interpolate.interp1d(z, k, bounds_error=False)

#
model_xav = ds.XC.isel(time=0, j=0).mean(dim='i').values
model_yav = ds.YC.isel(time=0, i=0).mean(dim='j').values
xmooring = model_xav # default lat/lon is the center of the domain
ymooring = model_yav
zmooring_TS = [-1, -10, -50, -100] # depth of T/S instruments
zmooring_UV = [-1, -10, -50, -100] # depth of U/V instruments

#
# ds.Theta.sel(k=1)
# ds.Theta.sel(k=1)
ki = f_z(zmooring_TS[0])
ii = f_x(xmooring)
ji = f_y(ymooring)
xo = ds.interp(i=ii, j=ji, k=ki)
xo.Theta.plot.line(x='time')
sdata.Theta.where(sdata.dep == sdata.dep.max(), drop=True).plot.line(x='time')
sgridded.Theta.isel(depth=0).plot()

In [ ]:
# test if the above regridding is correct by plotting the data at one level as a time series
# YES.

# shallowest (max because negative)
zpl = np.max(sdata['dep'].data)
print(zpl)

# select data at one depth:
this_var_onedep = sdata[vbl].where(sdata.dep == zpl, drop=True)

# fixed data at the same depth
this_var_fixed_onedep = this_var_fix[0,:]


# # deepest
# zpl = np.min(sdata['dep'].data)
# this_var_onedep = sdata[vbl].where(sdata.dep == zpl, drop=True)
# this_var_reshaped_onedep = this_var_reshape[494,:]
# print(zpl)


tpl = np.linspace(0,np.max(sdata['time']),len(this_var_reshaped_onedep))


plt.plot(sdata['time'].where(sdata.dep == zpl, drop=True).data, this_var_onedep.data,'.')
plt.plot(tpl,this_var_fixed_onedep,'r-.')

plt.show()

In [16]:
# --------------------------------------------------------------------
# USER INPUTS:
# --------------------------------------------------------------------
import datetime
# specify region from this list:
# WesternMed  ROAM_MIZ  NewCaledonia  NWPacific  BassStrait  RockallTrough  ACC_SMST
# MarmaraSea  LabradorSea  CapeBasin
RegionName = 'ACC_SMST' 

# specify date range as start date & number of days.
start_date = datetime.date(2012,1,1)
ndays = 31
# ndays = 2
# ndays = 10

# directory where data files are stored
# datadir = './data/' # default is ./data
datadir = '/data1/adac/mitgcm/netcdf/' + RegionName + '/'
outputdir = '/data1/adac/osse_output/' + RegionName + '/'
argodir = '/data1/argo/gridded/' # store argo climatology here

# optional details for sampling (if not specified, reasonable defaults will be used)
sampling_details = {
    'SAMPLING_STRATEGY' : 'sim_uctd', # options: sim_glider, sim_uctd or trajectory_file.add: mooring, ASV
#     'SAMPLING_STRATEGY' : 'sim_mooring', # options: sim_glider, sim_uctd or trajectory_file.add: mooring, ASV
    'PATTERN' : 'lawnmower', # back-forth or lawnmower 
    'zrange' : [-1, -1000],  # depth range of T/S profiles (down is negative). * add U/V range? *
    #'hspeed' : 0.25,  # platform horizontal speed in m/s
    #'vspeed' : 0.1, # platform vertical (profile) speed in m/s 
    #'trajectory_file' : None, # if SAMPLING_STRATEGY = 'trajectory_file', specify trajectory file
    #'AT_END' : 'repeat' # behaviour at and of trajectory: 'reverse', 'repeat' or 'terminate'. (could also 'restart'?)
}




sampling_details['SAMPLING_STRATEGY']
end_date = start_date + datetime.timedelta(days=ndays)
 
filename_out = (f'OSSE_{RegionName}_{sampling_details["SAMPLING_STRATEGY"]}_{start_date}_to_{end_date}')
# filename_out = (f'OSSE_{RegionName}_{sampling_details["SAMPLING_STRATEGY"]}')

print(filename_out)

OSSE_ACC_SMST_sim_uctd_2012-01-01_to_2012-02-01
